![Coleridge Initiative Logo](images/CI_horizontal.png)

<center>
    <span style="font-size: 1.5em;">
        <a href='https://www.coleridgeinitiative.org'>Website</a>
    </span>
</center>

Contributors: Ghani, Rayid, Frauke Kreuter, Julia Lane, Brian Kim, Adrianne Bradford, Alex Engler, Nicolas Guetta Jeanrenaud, Graham Henke, Daniela Hochfellner, Clayton Hunter, Avishek Kumar, Jonathan Morgan, Ekaterina Levitskaya.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Python-Setup" data-toc-modified-id="Python-Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Python Setup</a></span></li><li><span><a href="#Data-Import" data-toc-modified-id="Data-Import-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data Import</a></span></li><li><span><a href="#Record-Linkage" data-toc-modified-id="Record-Linkage-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Record Linkage</a></span><ul class="toc-item"><li><span><a href="#Indexing" data-toc-modified-id="Indexing-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Indexing</a></span></li></ul></li><li><span><a href="#Record-Comparison" data-toc-modified-id="Record-Comparison-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Record Comparison</a></span><ul class="toc-item"><li><span><a href="#Classification" data-toc-modified-id="Classification-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Classification</a></span></li></ul></li><li><span><a href="#References-and-Further-Readings" data-toc-modified-id="References-and-Further-Readings-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>References and Further Readings</a></span><ul class="toc-item"><li><span><a href="#Record-Linkage" data-toc-modified-id="Record-Linkage-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Record Linkage</a></span></li><li><span><a href="#Record-Linkage-Python-package" data-toc-modified-id="Record-Linkage-Python-package-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Record Linkage Python package</a></span></li><li><span><a href="#String-Comparators" data-toc-modified-id="String-Comparators-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>String Comparators</a></span></li><li><span><a href="#Fellegi-Sunter-Record-Linkage" data-toc-modified-id="Fellegi-Sunter-Record-Linkage-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Fellegi-Sunter Record Linkage</a></span></li></ul></li></ul></div>

## Python Setup

In [83]:
# data import and manipulation
import pandas as pd

# record linkage and preprocessing
import recordlinkage as rl

## Data Import

Read-in previously cleaned datasets.

In [88]:
# Read-in files with cleaned inventors (PatentsView) and principal investigators (FedRePORTER) names
inventors = pd.read_csv('inventors_cleaned.csv')
pi = pd.read_csv('pi_cleaned.csv')

In [89]:
# View first rows of the table
inventors.head()

,patent_id,year,inventor_id,name_first,name_last,city,state,inventor_country,name_middle,phonetic_first,phonetic_last,phonetic_middle
0,9980641,2018,5321501-6,joel,schuman,PITTSBURGH,PA,US,s,JAL,SANAN,S
1,9980641,2018,9980641-3,lawrence,kagemann,PITTSBURGH,PA,US,edward,LARANC,CAGANAN,EDWAD
2,9980645,2018,9037248-2,brian,wodlinger,PITTSBURGH,PA,US,NaN,BRAN,WADLANGAR,NaN
3,9981048,2018,9981048-1,ira,fox,PITTSBURGH,PA,US,jacob,IR,FAX,JACAB
4,9981048,2018,9981048-3,taichiro,nishikawa,PITTSBURGH,PA,US,n,TACAR,NASAC,N


In [90]:
# View first rows of the table
pi.head()

,PROJECT_ID,CONTACT_PI_PROJECT_LEADER,OTHER_PIS,city,state,ORGANIZATION_COUNTRY,name_first,name_last,name_middle,phonetic_first,phonetic_last,phonetic_middle
0,121219,poirer david,NaN,TUCSON,AZ,UNITED STATES,david,poirer,NaN,DAVAD,PARAR,NaN
1,121220,heyward ann o,NaN,BROOK PARK,OH,UNITED STATES,ann,heyward,o,AN,HAYWAD,O
2,121221,kalogera vassiliki,"LUIJTEN, ERIK",EVANSTON,IL,UNITED STATES,vassiliki,kalogera,NaN,VASALAC,CALAGAR,NaN
3,121222,guilderson thomas p,NaN,SANTA CRUZ,CA,UNITED STATES,thomas,guilderson,p,TAN,GALDARSAN,P
4,121223,hayes gillian,"TENTORI, MONICA",IRVINE,CA,UNITED STATES,gillian,hayes,NaN,GALAN,HAY,NaN


In [87]:
# Read-in files with cleaned organization names in PatentsView (assignees) and FedRePORTER (organizations)
assignees = pd.read_csv('assignees_cleaned.csv')
organizations = pd.read_csv('organizations_cleaned.csv')

In [8]:
# View first rows of the table
assignees.head()

,patent_id,org_name,city,state,org_country,phonetic_name
0,9992413,raytheoncompany,LEXINGTON,MA,US,RAYTANCANPANY
1,RE45718,bostonscientificcorporation,LEXINGTON,MA,US,BASTANSCANTAFACARPARATAN
2,9992413,raytheoncompany,WALTHAM,MA,US,RAYTANCANPANY
3,9988932,honeywellinternationalinc,WALTHAM,MA,US,HANAYWALANTARNATANALANC
4,9991578,honeywellinternationalinc,WALTHAM,MA,US,HANAYWALANTARNATANALANC


In [9]:
# View first rows of the table
organizations.head()

,PROJECT_ID,org_name,city,state,ORGANIZATION_COUNTRY,phonetic_name
0,121219,universityofarizona,TUCSON,AZ,UNITED STATES,UNAVARSATYAFARASAN
1,121220,ohioaerospaceinstitute,BROOK PARK,OH,UNITED STATES,OHARASPACANSTATAT
2,121221,northwesternuniversity,EVANSTON,IL,UNITED STATES,NARTWASTARNANAVARSATY
3,121222,universityofcaliforniasantacruz,SANTA CRUZ,CA,UNITED STATES,UNAVARSATYAFCALAFARNASANTACR
4,121223,universityofcaliforniairvine,IRVINE,CA,UNITED STATES,UNAVARSATYAFCALAFARNARVAN


## Record Linkage

The `recordlinkage` package is a quite powerful tool for you to use when you want to link records within a dataset or across multiple datasets. We've already done some pre-processing and then tried deterministic matching.

However, as we have seen in the previous notebook, we might want to consider how strict we want our matching to be. For example, we want to make sure that we catch any typos or common misspellings, but we want to avoid relaxing the matching condition to the point that anything will match anything. 

### Indexing

Indexing allows us to create candidate links, which basically means identifying pairs of data rows which might refer to the same real world entity. This is also called the comparison space (matrix). There are different ways to index data. The easiest is to create a full index and consider every pair a match. This is also the least efficient method, because we will be comparing every row of one dataset with every row of the other dataset.

If we had 10,000 records in data frame A and 100,000 records in data frame B, we would have 1,000,000,000 candidate links. You can see that comparing over a full index is getting inefficient when working with big data.

We can do better if we actually include our knowledge about the data to eliminate bad link from the start. This can be done through blocking. The `recordlinkage` package gives you multiple options for this. For example, you can block by using variables, which means only links exactly equal on specified values will be kept. 

Here we will start by blocking on `city` and `state`, to narrow down the number of candidate links.

You can try and see how the number of candidate links change when blocking on more or less variables.

In [91]:
indexerBL = rl.BlockIndex(on=['city', 'state'])
candidate_links = indexerBL.index(inventors, pi)

In [92]:
len(candidate_links)

1924057

In [93]:
candidate_links[:10]

MultiIndex([(0,  85),
            (0, 214),
            (0, 305),
            (0, 317),
            (0, 468),
            (0, 477),
            (0, 530),
            (0, 723),
            (0, 836),
            (0, 885)],
           )

Let's check the first pair of candidate links blocked on city and state: (0, 85)

In [94]:
inventors.iloc[0]

patent_id              9980641
year                      2018
inventor_id          5321501-6
name_first                joel
name_last              schuman
city                PITTSBURGH
state                       PA
inventor_country            US
name_middle                  s
phonetic_first             JAL
phonetic_last            SANAN
phonetic_middle              S
Name: 0, dtype: object

In [95]:
pi.iloc[85]

PROJECT_ID                           121304
 CONTACT_PI_PROJECT_LEADER          yang ge
 OTHER_PIS                              NaN
city                             PITTSBURGH
state                                    PA
 ORGANIZATION_COUNTRY         UNITED STATES
name_first                               ge
name_last                              yang
name_middle                             NaN
phonetic_first                            G
phonetic_last                          YANG
phonetic_middle                         NaN
Name: 85, dtype: object

In [101]:
# Now, in addition to blocking on city and state, we can also try blocking on first name
indexerBL = rl.BlockIndex(on=['name_first','city', 'state'])
candidate_links = indexerBL.index(inventors, pi)

In [102]:
len(candidate_links)

6931

## Record Comparison

After you have created a set of candidate links, you’re ready to begin comparing the records associated with each candidate link. In `recordlinkage` package you must initiate a Compare object prior to performing any comparison functionality between records. This object stores both dataframes, the candidate links, and a vector containing comparison results. Further, the Compare object contains the methods for performing comparisons. The code block below initializes the comparison object.

In [106]:
# Initiate compare object 
compare_cl = rl.Compare()

Currently there are five specific comparison methods within recordlinkage: `Compare.exact()`, `Compare.string()`, `Compare.numeric()`, `Compare.geo()`, and `Compare.date()`. 

The `Compare.exact()` method is simple: if two values are an exact match a comparison score of 1 is returned, otherwise 0 is retured. 

The `Compare.string()` method is a bit more complicated and generates a score based on well-known string-comparison algorithms (for this example, Levenshtein or Jaro Winkler).

The Python `recordlinkage` toolkit uses the `jellyfish` package for the Jaro, Jaro-Winkler, Levenshtein and Damerau-Levenshtein algorithms: https://jellyfish.readthedocs.io/en/latest/comparison.html.

For this example, Jaro-Winkler distance is used (specifically developed with record linkage applications in mind, faster to compute) - words with more characters in common have a higher Jaro-Winkler value than those with fewer characters in common. The Jaro–Winkler distance gives more favorable ratings to strings that match from the beginning. The output value is normalized to fall between 0 (complete dissimilar strirngs) and 1 (exact match on strings).

As you remember, we already did an exact matching on `city` and `state`, when we did the blocking above and created the candidate links.

We will use the string method to compare the organization names and their phonetic transcriptions.

We need to specify the respective columns with organization names in both datasets, the method, and the threshold. In this case, for all strings that have more than 85% in similarity, according to the Jaro-Winkler distance, a 1 will be returned, and otherwise 0.

In [221]:
# Initiate compare object 
compare_cl = rl.Compare()

In [222]:
compare_cl.string('name_first', 'name_first', method='jarowinkler', threshold=0.85, label='name_first')
compare_cl.string('name_last', 'name_last', method='jarowinkler', threshold=0.85, label='name_last')

#compare_cl.exact('state', 'state', label='state')

<Compare>

The comparing of record pairs starts when the `compute` method is called.

In [223]:
indexerBL = rl.BlockIndex(on=['city', 'state'])
candidate_links = indexerBL.index(inventors, pi)

In [224]:
len(candidate_links)

1924057

In [225]:
## All attribute comparisons are stored in a DataFrame with horizontally the features and vertically the record pairs.
features = compare_cl.compute(candidate_links, inventors, pi)

In [226]:
features.head(7)

name_first  name_last
0 85          0.0        0.0
  214         0.0        0.0
  305         0.0        0.0
  317         0.0        0.0
  468         0.0        0.0
  477         0.0        0.0
  530         0.0        1.0

In [227]:
features.tail()

name_first  name_last
3995 17363         0.0        0.0
     52988         1.0        1.0
     61613         0.0        0.0
     72322         1.0        1.0
     75295         0.0        0.0

### Classification

Let's check how many records we get where one or more comparison attributes match.

In [228]:
## Simple Classification: Check for how many attributes records are identical by summing the comparison results.
features.sum(axis=1).value_counts().sort_index(ascending=False)

2.0        558
1.0      14257
0.0    1909242
dtype: int64

We can make a decision now, and consider matches all those records which matched on all attributes in our case.

In [229]:
matches = features[features.sum(axis=1) == 2]
print(len(matches))

558


Remember that for these matches we had an exact match on `city` and `state`, and more than 80% in similarity based on organization `first name` and `last name`

In [230]:
matches.head()

name_first  name_last
0 23494         1.0        1.0
3 56152         1.0        1.0
  77251         1.0        1.0
8 2727          1.0        1.0
  13392         1.0        1.0

Now let's merge these matches back to original dataframes.

Our `matches` dataframe has MultiIndex - two indices to the left which correspond to the `inventor` table and `pi` table respectively.

We can access each matching pair individually, for example, the first one:

In [231]:
matches.index[0]

(0, 23494)

We can also do the following: first, put all the indices for the `inventors` table.

In [232]:
matches.index[0][0]

0

We will pull all corresponding rows from the `inventors` table.

In [233]:
inventors_results = []  # Create an empty list

for match in matches.index:  # For every pair in matches (index)
    df = pd.DataFrame(inventors.loc[[match[0]]])  # Get the location in the original table, convert to dataframe
    inventors_results.append(df)

In [234]:
inventors_results[0]

,patent_id,year,inventor_id,name_first,name_last,city,state,inventor_country,name_middle,phonetic_first,phonetic_last,phonetic_middle
0,9980641,2018,5321501-6,joel,schuman,PITTSBURGH,PA,US,s,JAL,SANAN,S


Now we concatenate the list of dataframes into one dataframe.

In [235]:
inventors_concat = pd.concat(inventors_results)

In [236]:
inventors_concat.head()

,patent_id,year,inventor_id,name_first,name_last,city,state,inventor_country,name_middle,phonetic_first,phonetic_last,phonetic_middle
0,9980641,2018,5321501-6,joel,schuman,PITTSBURGH,PA,US,s,JAL,SANAN,S
3,9981048,2018,9981048-1,ira,fox,PITTSBURGH,PA,US,jacob,IR,FAX,JACAB
3,9981048,2018,9981048-1,ira,fox,PITTSBURGH,PA,US,jacob,IR,FAX,JACAB
8,9982070,2018,5077078-2,krzysztof,matyjaszewski,PITTSBURGH,PA,US,a,CRSYSTAF,MATYJASAESC,A
8,9982070,2018,5077078-2,krzysztof,matyjaszewski,PITTSBURGH,PA,US,a,CRSYSTAF,MATYJASAESC,A


We do the same for the `pi` table.

In [237]:
pi_results = []  # Create an empty list

for match in matches.index:  # For every pair in matches (index)
    df = pd.DataFrame(pi.loc[[match[1]]])  # Get the location in the original table, convert to dataframe
    pi_results.append(df)
    
pi_concat = pd.concat(pi_results)

In [238]:
pi_concat.head()

,PROJECT_ID,CONTACT_PI_PROJECT_LEADER,OTHER_PIS,city,state,ORGANIZATION_COUNTRY,name_first,name_last,name_middle,phonetic_first,phonetic_last,phonetic_middle
23494,289911,schuman joel s,NaN,PITTSBURGH,PA,UNITED STATES,joel,schuman,s,JAL,SANAN,S
56152,339486,fox ira j,NaN,PITTSBURGH,PA,UNITED STATES,ira,fox,j,IR,FAX,J
77251,393971,fox ira j,NaN,PITTSBURGH,PA,UNITED STATES,ira,fox,j,IR,FAX,J
2727,123951,matyjaszewski krzysztof,NaN,PITTSBURGH,PA,UNITED STATES,krzysztof,matyjaszewski,NaN,CRSYSTAF,MATYJASAESC,NaN
13392,134632,matyjaszewski krzysztof,NaN,PITTSBURGH,PA,UNITED STATES,krzysztof,matyjaszewski,NaN,CRSYSTAF,MATYJASAESC,NaN


Now we need to combine two tables on the index - notice that our tables right now have indices from the original tables. We can reset the index using `.reset_index()`.

In [239]:
inventors_concat = inventors_concat.reset_index()
pi_concat = pi_concat.reset_index()

Now our tables have the same index on which we can combine two tables.

In [240]:
inventors_concat.head()

,index,patent_id,year,inventor_id,name_first,name_last,city,state,inventor_country,name_middle,phonetic_first,phonetic_last,phonetic_middle
0,0,9980641,2018,5321501-6,joel,schuman,PITTSBURGH,PA,US,s,JAL,SANAN,S
1,3,9981048,2018,9981048-1,ira,fox,PITTSBURGH,PA,US,jacob,IR,FAX,JACAB
2,3,9981048,2018,9981048-1,ira,fox,PITTSBURGH,PA,US,jacob,IR,FAX,JACAB
3,8,9982070,2018,5077078-2,krzysztof,matyjaszewski,PITTSBURGH,PA,US,a,CRSYSTAF,MATYJASAESC,A
4,8,9982070,2018,5077078-2,krzysztof,matyjaszewski,PITTSBURGH,PA,US,a,CRSYSTAF,MATYJASAESC,A


In [241]:
pi_concat.head()

,index,PROJECT_ID,CONTACT_PI_PROJECT_LEADER,OTHER_PIS,city,state,ORGANIZATION_COUNTRY,name_first,name_last,name_middle,phonetic_first,phonetic_last,phonetic_middle
0,23494,289911,schuman joel s,NaN,PITTSBURGH,PA,UNITED STATES,joel,schuman,s,JAL,SANAN,S
1,56152,339486,fox ira j,NaN,PITTSBURGH,PA,UNITED STATES,ira,fox,j,IR,FAX,J
2,77251,393971,fox ira j,NaN,PITTSBURGH,PA,UNITED STATES,ira,fox,j,IR,FAX,J
3,2727,123951,matyjaszewski krzysztof,NaN,PITTSBURGH,PA,UNITED STATES,krzysztof,matyjaszewski,NaN,CRSYSTAF,MATYJASAESC,NaN
4,13392,134632,matyjaszewski krzysztof,NaN,PITTSBURGH,PA,UNITED STATES,krzysztof,matyjaszewski,NaN,CRSYSTAF,MATYJASAESC,NaN


In [242]:
# Drop the old index column
inventors_concat = inventors_concat.drop(columns=['index'])
pi_concat = pi_concat.drop(columns=['index'])

In [243]:
# Drop other not relevant columns
inventors_concat = inventors_concat.drop(columns=['inventor_country'])
pi_concat = pi_concat.drop(columns=[' ORGANIZATION_COUNTRY'])

Now we concatenate these two tables using `.concat()`.

In [244]:
matched = pd.concat([inventors_concat, pi_concat], axis=1)  # Specify axis=1 to concatenate horizontally

In [250]:
matched[14:20]

,patent_id,year,inventor_id,name_first,name_last,city,state,name_middle,phonetic_first,phonetic_last,...,CONTACT_PI_PROJECT_LEADER,OTHER_PIS,city,state,name_first,name_last,name_middle,phonetic_first,phonetic_last,phonetic_middle
14,9987329,2018,9020600-3,mark,rosenblatt,NEW YORK,NY,NaN,MARC,RASANBLAT,...,rosenblatt mark i,NaN,NEW YORK,NY,mark,rosenblatt,i,MARC,RASANBLAT,I
15,9989451,2018,6055106-1,david,grier,NEW YORK,NY,g,DAVAD,GRAR,...,gruber david,"GREER, EDYTA",NEW YORK,NY,david,gruber,NaN,DAVAD,GRABAR,NaN
16,9989451,2018,6055106-1,david,grier,NEW YORK,NY,g,DAVAD,GRAR,...,gruber david,"MANFRINO, CARRIE; PIERIBONE, VINCENT",NEW YORK,NY,david,gruber,NaN,DAVAD,GRABAR,NaN
17,9992222,2018,6185678-3,angelos,keromytis,NEW YORK,NY,d,ANGAL,CARANYT,...,keromytis angelos d,NaN,NEW YORK,NY,angelos,keromytis,d,ANGAL,CARANYT,D
18,9993465,2018,5326659-1,yong,liu,NEW YORK,NY,NaN,YANG,L,...,li yihong,NaN,NEW YORK,NY,yihong,li,NaN,YAHANG,L,NaN
19,9996743,2018,4876455-3,shree,nayar,NEW YORK,NY,k,SRY,NAYAR,...,nayar shree k,NaN,NEW YORK,NY,shree,nayar,k,SRY,NAYAR,C


Now that we have merged our matches together, examine them. Remember that we matched our strings on 85% similarity, according to the Jaro-Winkler distance,  and we blocked on city and state. Try using a different threshold. You can also use a different string-matching algorithm (please see below in References).

<span style="color:red">**Checkpoint: Record Linkage Decisions**</span>

What are some decisinos we had to make as we went through the record linkage process above? What if we had made different choices instead?

Try doing the record linkage with a few different options and see how many matches you get as you vary the approach. For example, try different string-matching algorithms or thresholds.

In [ ]:
# your code here...





## References and Further Readings

### Record Linkage

Lane, Julia, Ian Foster, Rayid Ghani, Ron S. Jarmin, Frauke Kreuter (editors), Big Data and Social Science: A Practical Guide to Methods and Tools, Chapman and Hall/CRC Press, 2016. https://coleridge-initiative.github.io/big-data-and-social-science/chap-link.html

### Record Linkage Python package
* `recordlinkage` Python package: https://recordlinkage.readthedocs.io/en/latest/index.html
    - Comparing records: https://recordlinkage.readthedocs.io/en/latest/ref-compare.html
    - Classification:
        - https://recordlinkage.readthedocs.io/en/latest/ref-classifiers.html,
        - https://recordlinkage.readthedocs.io/en/latest/notebooks/classifiers.html

### String Comparators

* GitHub page of `jellyfish`: https://github.com/jamesturk/jellyfish
* Descriptions of distances in `jellyfish`: https://jellyfish.readthedocs.io/en/latest/comparison.html
* Different distances that measure the differences between strings:
    - Levenshtein distance: https://en.wikipedia.org/wiki/Levenshtein_distance
    - Damerau–Levenshtein distance: https://en.wikipedia.org/wiki/Damerau%E2%80%93Levenshtein_distance
    - Jaro–Winkler distance: https://en.wikipedia.org/wiki/Jaro%E2%80%93Winkler_distance
    - Hamming distance: https://en.wikipedia.org/wiki/Hamming_distance
    - Match rating approach: https://en.wikipedia.org/wiki/Match_rating_approach

### Fellegi-Sunter Record Linkage 

* Introduction to Probabilistic Record Linkage: http://www.bristol.ac.uk/media-library/sites/cmm/migrated/documents/problinkage.pdf
* Paper Review: https://www.cs.umd.edu/class/spring2012/cmsc828L/Papers/HerzogEtWires10.pdf

